In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
import time
from datetime import datetime, timedelta
import shutil
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import statsmodels.tsa.api as tsa
import torch
from pandarallel import pandarallel
import seaborn as sns
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.preprocessing import StandardScaler



In [2]:
X_order = np.load('tmp/sorted_order.npy')
X_page = np.load('tmp/sorted_page_day60.npy')
X_metric = np.load('tmp/sorted_metric.npy')
y = np.load('tmp/sorted_label.npy')

In [4]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--d_in', type=int, default=5, help='input size')
parser.add_argument('--d_emb', type=int, default=32, help='embedding size')
parser.add_argument('--d_model', type=int, default=256, help='dimension of model')
parser.add_argument('--d_tar', type=int, default=2, help='dimension of output')
args_1 = parser.parse_args(args = [])

In [5]:
import argparse
parser = argparse.ArgumentParser()
d_in = X_page.shape[2] * X_page.shape[3]
parser.add_argument('--d_in', type=int, default=d_in, help='input size')
parser.add_argument('--d_emb', type=int, default=32, help='embedding size')
parser.add_argument('--d_model', type=int, default=256, help='dimension of model')
parser.add_argument('--d_tar', type=int, default=2, help='dimension of output')
args_2 = parser.parse_args(args = [])

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('--args_1', type=argparse.Namespace, default=args_1)
parser.add_argument('--args_2', type=argparse.Namespace, default=args_2)
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=20, help='training epochs')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--val_rate', type=float, default=0.1, help='validation rate')
args_train_1 = parser.parse_args(args = [])

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--args_1', type=argparse.Namespace, default=args_1)
parser.add_argument('--args_2', type=argparse.Namespace, default=args_2)
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=30, help='training epochs')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--val_rate', type=float, default=0.1, help='validation rate')
args_train_2 = parser.parse_args(args = [])

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--enc_size', type=int, default=18, help='encoder input size')
parser.add_argument('--d_model', type=int, default=128, help='dimension of model')
parser.add_argument('--e_layers', type=int, default=3, help='num of encoder layers')
parser.add_argument('--seq_len', type=int, default=60, help='input sequence length')
parser.add_argument('--dropout', type=float, default=0.3, help='dropout')
parser.add_argument('--n_head', type=int, default=8, help='num of heads')
parser.add_argument('--ffn_hidden', type=int, default=512, help='dimension of hidden layer')

parser.add_argument('--clf_choose', type=int, default=2, help='which classifier to use')
parser.add_argument('--batch_size', type=int, default=32, help='batch size')
parser.add_argument('--epochs', type=int, default=15, help='training epochs')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--val_rate', type=float, default=0.1, help='validation rate')
args_T = parser.parse_args(args = [])

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=32, help='batch size')
parser.add_argument('--epochs', type=int, default=10, help='training epochs')
parser.add_argument('--lr', type=float, default=1e-4, help='learning rate')
parser.add_argument('--val_rate', type=float, default=0.1, help='validation rate')
args_finetune = parser.parse_args(args = [])

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--args_1', type=argparse.Namespace, default=args_1)
parser.add_argument('--args_2', type=argparse.Namespace, default=args_2)
parser.add_argument('--args_train_1', type=argparse.Namespace, default=args_train_1)
parser.add_argument('--args_train_2', type=argparse.Namespace, default=args_train_2)
parser.add_argument('--args_T', type=argparse.Namespace, default=args_T)
parser.add_argument('--args_finetune', type=argparse.Namespace, default=args_finetune)
parser.add_argument('--verbose', type=bool, default=True)
parser.add_argument('--weight_decay', type=float, default=0)
args = parser.parse_args(args = [])

In [ ]:
%run models/CloudChurn.py
k = 10
from sklearn.model_selection import StratifiedKFold
f1_list = []
auc_list = []
precision_list = []
acc_list = []
sfolder = StratifiedKFold(n_splits=k, shuffle=True)
fold_count = 1
for train_index, test_index in sfolder.split(np.arange(y.shape[0]), y):
    print("--------------------- fold {} ----------------------".format(fold_count))
    fold_count += 1
    X_train_order = []
    X_test_order = []
    X_train_page = []
    X_test_page = []
    X_train_metric = []
    X_test_metric = []
    y_train = []
    y_test = []


    for id in train_index:
        X_train_order.append(X_order[id])
        X_train_page.append(X_page[id])
        X_train_metric.append(X_metric[id])
        y_train.append(y[id])

    for id in test_index:
        X_test_order.append(X_order[id])
        X_test_page.append(X_page[id])
        X_test_metric.append(X_metric[id])
        y_test.append(y[id])

    X_train_order = np.array(X_train_order)
    X_train_page = np.array(X_train_page)
    X_train_metric = np.array(X_train_metric)
    X_test_order = np.array(X_test_order)
    X_test_page = np.array(X_test_page)
    X_test_metric = np.array(X_test_metric)

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    scale = False
    if scale:
        from sklearn.preprocessing import MinMaxScaler
        min_max_scaler = MinMaxScaler()
        X_train_order = min_max_scaler.fit_transform(X_train_order)
        X_test_order = min_max_scaler.transform(X_test_order)
    
    X_train_order_t = torch.tensor(X_train_order, dtype=torch.float32)
    X_train_page_t = torch.tensor(X_train_page, dtype=torch.float32)
    X_train_metric_t = torch.tensor(X_train_metric, dtype=torch.float32)

    X_test_order_t = torch.tensor(X_test_order, dtype=torch.float32)
    X_test_page_t = torch.tensor(X_test_page, dtype=torch.float32)
    X_test_metric_t = torch.tensor(X_test_metric, dtype=torch.float32)

    y_train_t = torch.tensor(y_train, dtype=torch.long)
    y_test_t = torch.tensor(y_test, dtype=torch.long)
    
    
    # --------------- CloudChurn Test ---------------------------------------------------
    args.args_T.clf_choose = 2
    args.verbose = False
    
    model = CloudChurn(args)
    model.model_train(X_train_order_t, X_train_page_t, X_train_metric_t, y_train_t)
    f1, auc = model.get_metric(X_test_order_t, X_test_page_t, X_test_metric_t, y_test_t)
    model.model_test(X_test_order_t, X_test_page_t, X_test_metric_t, y_test_t)
    report = model.predict_report(X_test_order_t, X_test_page_t, X_test_metric_t, y_test_t)
    precision = float(report[127:132]) # precision
    acc = float(report[202:207]) # accuracy
    precision_list.append(precision)
    acc_list.append(acc)
    f1_list.append(f1)
    auc_list.append(auc)
    

In [ ]:
f1_list

In [ ]:
auc_list

In [ ]:
np.array(precision_list).mean()

In [ ]:
np.array(f1_list)[:,2].mean()

In [ ]:
np.array(acc_list).mean()

In [ ]:
np.array(auc_list)[:,2].mean()